In [1]:
import streamlit as st
import tempfile
import os
from langchain_community.vectorstores import FAISS
# from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, AzureChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings


In [34]:
def load_document(tmp_file_path):
  # with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
  #     tmp_file.write(file.getvalue())
  #     tmp_file_path = tmp_file.name

  loader = PyPDFLoader(tmp_file_path)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
  docs = text_splitter.split_documents(documents)
  
  # os.unlink(tmp_file_path)
  return docs

def create_vector_db(docs, path):
  embedding_function = AzureOpenAIEmbeddings(
            openai_api_type = "azure",
            openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
            deployment = "text-embedding-ada-002",
            model = "text-embedding-ada-002"
        )
  db = FAISS.from_documents(docs, embedding_function)
  db.save_local(path)
  return db

def load_vector_db(path):
  embedding_function = AzureOpenAIEmbeddings(
            openai_api_type = "azure",
            openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
            deployment = "text-embedding-ada-002",
            model = "text-embedding-ada-002"
        )
  return FAISS.load_local(path, embedding_function, allow_dangerous_deserialization=True)

def analyze_eligibility(rfp_content, proposal_content):
  llm= AzureChatOpenAI(
                openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
                openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
                azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
                temperature = 0,
                
            )

  template = """
  You are an AI contract analyzer. Your task is to compare the eligibility criteria listed in a Request for Proposal (RFP) with the details provided in a proposal.

  RFP Content:
  {rfp_content}

  Proposal Content:
  {proposal_content}

  For each eligibility criterion in the RFP, provide the following:

  Eligibility Criterion: [Insert eligibility criterion from RFP]
  Eligibility Met (Yes/No): [Yes/No]
  Reason: [Provide a detailed explanation of how the eligibility criterion is met or not met based on the proposal]

  Please provide your analysis in a clear, structured format.
  """

  prompt = PromptTemplate(input_variables=["rfp_content", "proposal_content"], template=template)
  chain = LLMChain(llm=llm, prompt=prompt)
  
  return chain.run(rfp_content=rfp_content, proposal_content=proposal_content)


In [36]:
rfp_file ="../data/tendernotice-1-11713759672.pdf"
# proposal_file = st.file_uploader("Upload Proposal (PDF)", type="pdf")
# Load and process RFP
rfp_docs = load_document(rfp_file)
rfp_db = create_vector_db(rfp_docs, "../vectorstore/RFP")


# Load and process Proposal
proposal_file= "../data/Proposal for Conducting Market Study on Artificial Intelligence and Competition.pdf"
proposal_docs = load_document(proposal_file)
proposal_db = create_vector_db(proposal_docs, "../vectorstore/Proposals/")

In [64]:
# Retrieve relevant content
rfp_content = rfp_db.similarity_search("eligibility criteria", k=20)
proposal_content = proposal_db.similarity_search("company background and qualifications", k=20)

In [65]:
rfp_content

[Document(metadata={'source': '../data/tendernotice-1-11713759672.pdf', 'page': 11}, page_content='ii. CCI’s nominated Officers will be part of “Study Team” to ensure timely completion \nof the study and quality assessment of the submissions made by the Agency.  \n \n \n \nVIII.   PRE -QUALIFICATION/ ELIGIBILITY CRITERIA  \n \nThe pre -qualification/ eligibility criteria for the agencies are provided in the table  below:  \n \nS. No.  Pre-Qualification Criteria  Proof Required  \n1. A minimum  of 5 years of existence  \n Copy of Certificate of \nIncorporation/ Registration/ MoA, \nas applicable.  \n2. The Agency should have a minimum Annual \nTurnover of Rs. 50 Cr. during each of the  3 \nfinancial years  i.e., FY 2020 -21, FY 2021 -22 \nand FY 2022 -23 Copies of  financial statements \nduly audited by statutory auditors . \nIn the absence of the same,  \ncertificates issued by chartered \naccountants  for the relevant years , \nas per Annexure -1 \n3. The Agency should have co mpleted

In [66]:
# Combine retrieved content
rfp_text = " ".join([doc.page_content for doc in rfp_content])
proposal_text = " ".join([doc.page_content for doc in proposal_content])

In [67]:
rfp_text

'ii. CCI’s nominated Officers will be part of “Study Team” to ensure timely completion \nof the study and quality assessment of the submissions made by the Agency.  \n \n \n \nVIII.   PRE -QUALIFICATION/ ELIGIBILITY CRITERIA  \n \nThe pre -qualification/ eligibility criteria for the agencies are provided in the table  below:  \n \nS. No.  Pre-Qualification Criteria  Proof Required  \n1. A minimum  of 5 years of existence  \n Copy of Certificate of \nIncorporation/ Registration/ MoA, \nas applicable.  \n2. The Agency should have a minimum Annual \nTurnover of Rs. 50 Cr. during each of the  3 \nfinancial years  i.e., FY 2020 -21, FY 2021 -22 \nand FY 2022 -23 Copies of  financial statements \nduly audited by statutory auditors . \nIn the absence of the same,  \ncertificates issued by chartered \naccountants  for the relevant years , \nas per Annexure -1 \n3. The Agency should have co mpleted  a \nminimum of 5 studies/ projects (AI \nrelated/tech -policy related/competition \nrelated/sect

In [68]:
llm= AzureChatOpenAI(
            openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
            openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
            temperature = 0,
            
        )

template = """
You are an AI contract analyzer. Your task is to Find all the eligibility criteria listed in a Request for Proposal (RFP).

RFP Content:
{rfp_text}

"""

prompt = PromptTemplate(input_variables=["rfp_text"], template=template)
chain = LLMChain(llm=llm, prompt=prompt)

rfp_result=chain.run(rfp_text=rfp_text)

from IPython.display import Markdown, display
display(Markdown(rfp_result))

The eligibility criteria listed in the Request for Proposal (RFP) are as follows:

1. **A minimum of 5 years of existence**
   - **Proof Required:** Copy of Certificate of Incorporation/Registration/MoA, as applicable.

2. **The Agency should have a minimum Annual Turnover of Rs. 50 Cr. during each of the 3 financial years i.e., FY 2020-21, FY 2021-22, and FY 2022-23**
   - **Proof Required:** Copies of financial statements duly audited by statutory auditors. In the absence of the same, certificates issued by chartered accountants for the relevant years, as per Annexure-1.

3. **The Agency should have completed a minimum of 5 studies/projects (AI related/tech-policy related/competition related/sectoral studies for policy purposes)**
   - **Proof Required:** Not explicitly mentioned in the provided text, but typically would include project completion reports or similar documentation.

4. **The Agency should not have been blacklisted by Central/State Government Departments/Undertakings/Court of Law**
   - **Proof Required:** No Conviction/Debarment Certificate duly signed by the Authorised Signatory of the Agency, as per Annexure-3.

5. **The agency should have a valid PAN and Service Tax/GST Registration in India (if applicable)**
   - **Proof Required:** Self-attested copy of PAN card and Service Tax/GST Registration certificate (if applicable).

In [69]:
llm= AzureChatOpenAI(
            openai_api_key = os.getenv("AZURE_OPENAI_API_KEY"),
            openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
            temperature = 0,
            
        )

template = """
You are an AI contract analyzer. Your task is to compare the eligibility criteria listed in a Request for Proposal (RFP) with the details provided in a proposal.

RFP Content:
{rfp_result}

Proposal Content:
{proposal_content}

For each eligibility criterion in the RFP, provide the following:

Eligibility Criterion: [Insert eligibility criterion from RFP]
Eligibility Met (Yes/No): [Yes/No]
Reason: [Provide a detailed explanation of how the eligibility criterion is met or not met based on the proposal]

Please provide your analysis in a clear, structured format.
"""

prompt = PromptTemplate(input_variables=["rfp_result", "proposal_content"], template=template)
chain = LLMChain(llm=llm, prompt=prompt)

result=chain.run(rfp_result=rfp_result, proposal_content=proposal_content)
display(Markdown(result))

Certainly! Below is the structured analysis comparing the eligibility criteria listed in the RFP with the details provided in the proposal.

### Eligibility Criterion 1: A minimum of 5 years of existence
**Eligibility Met (Yes/No):** Yes  
**Reason:** The proposal states that Quantum Analytics Consulting has over 15 years of global experience. Additionally, Annexure 1 includes the Certificate of Incorporation, which serves as proof of the company's existence for more than 5 years.

### Eligibility Criterion 2: The Agency should have a minimum Annual Turnover of Rs. 50 Cr. during each of the 3 financial years i.e., FY 2020-21, FY 2021-22, and FY 2022-23
**Eligibility Met (Yes/No):** No  
**Reason:** The proposal only includes financial statements for FY 2022-23 (Annexure 2). There is no mention or inclusion of financial statements or certificates for FY 2020-21 and FY 2021-22, which are required to meet this criterion.

### Eligibility Criterion 3: The Agency should have completed a minimum of 5 studies/projects (AI related/tech-policy related/competition related/sectoral studies for policy purposes)
**Eligibility Met (Yes/No):** Yes  
**Reason:** The proposal mentions that Quantum Analytics Consulting has conducted over 50 studies globally on technology's impact on market structures, including ten in AI and competition. Annexure 3 includes copies of completion certificates for relevant past projects, which supports this claim.

### Eligibility Criterion 4: The Agency should not have been blacklisted by Central/State Government Departments/Undertakings/Court of Law
**Eligibility Met (Yes/No):** No  
**Reason:** The proposal does not include a No Conviction/Debarment Certificate duly signed by the Authorised Signatory of the Agency, as per Annexure-3. This document is required to prove that the agency has not been blacklisted.

### Eligibility Criterion 5: The agency should have a valid PAN and Service Tax/GST Registration in India (if applicable)
**Eligibility Met (Yes/No):** No  
**Reason:** The proposal does not include a self-attested copy of the PAN card and Service Tax/GST Registration certificate. These documents are necessary to meet this eligibility criterion.

### Summary
Based on the analysis, Quantum Analytics Consulting meets the following eligibility criteria:
- Criterion 1: A minimum of 5 years of existence
- Criterion 3: The Agency should have completed a minimum of 5 studies/projects

However, the proposal does not meet the following criteria:
- Criterion 2: Minimum Annual Turnover of Rs. 50 Cr. during each of the 3 financial years
- Criterion 4: The Agency should not have been blacklisted
- Criterion 5: Valid PAN and Service Tax/GST Registration in India

The proposal is partially compliant with the RFP's eligibility criteria.